In [1]:
import dask
from dask.distributed import Client, LocalCluster, wait
dask.config.set(temporary_directory='D:\Vali\LTR\Temp')

cluster = LocalCluster(n_workers=2, threads_per_worker=4)
client = Client(cluster)
client.wait_for_workers(2)
print(f"View the dashboard: {cluster.dashboard_link}")

View the dashboard: http://127.0.0.1:8787/status


In [2]:
from dask.dataframe import read_csv

train_df = read_csv('train.csv')
validation_df = read_csv('validation.csv')

In [3]:
import pandas as pd
import numpy as np

qids_train = train_df.groupby("QID")["QID"].count().to_dask_array(lengths=True)
X_train = train_df.drop(["QID", "Relevance"], axis=1)
y_train = train_df["Relevance"]

In [4]:
qids_validation = validation_df.groupby("QID")["QID"].count().to_dask_array(lengths=True)
X_validation = validation_df.drop(["QID", "Relevance"], axis=1)
y_validation = validation_df["Relevance"]

In [5]:
import dask

NO_PARTITIONS = 32

N_queries = qids_train.size
chunks_q = []
chunks_data = []
sm = 0
for i in range(NO_PARTITIONS):
    if i == NO_PARTITIONS - 1:
        nr = N_queries - sum(chunks_q)
    else:
        nr = N_queries // NO_PARTITIONS
    chunks_q.append(nr)
    chunks_data.append(np.sum(qids_train[sm:sm+nr]))
    sm += nr
    
print(chunks_q)
chunks_data = dask.compute(chunks_data)[0]
print(chunks_data)

[591, 591, 591, 591, 591, 591, 591, 591, 591, 591, 591, 591, 591, 591, 591, 591, 591, 591, 591, 591, 591, 591, 591, 591, 591, 591, 591, 591, 591, 591, 591, 598]
[65744, 64697, 75526, 73786, 73147, 79304, 82082, 84755, 57169, 55347, 64085, 65294, 66972, 78010, 73710, 71660, 81323, 82248, 72533, 59056, 59134, 69316, 63770, 67815, 81608, 73715, 77549, 82334, 89803, 62003, 55173, 61628]


In [6]:
dx = X_train.to_dask_array(lengths=True)
dy = y_train.to_dask_array(lengths=True)

qids_train = qids_train.rechunk([tuple(chunks_q)])
dx = dx.rechunk([tuple(chunks_data), 136])
dy = dy.rechunk([tuple(chunks_data)])

In [7]:
N_queries = qids_validation.size
chunks_q = []
chunks_data = []
sm = 0
for i in range(NO_PARTITIONS):
    if i == NO_PARTITIONS - 1:
        nr = N_queries - sum(chunks_q)
    else:
        nr = N_queries // NO_PARTITIONS
    chunks_q.append(nr)
    chunks_data.append(np.sum(qids_validation[sm:sm+nr]))
    sm += nr
    
print(chunks_q)
chunks_data = dask.compute(chunks_data)[0]
print(chunks_data)

[197, 197, 197, 197, 197, 197, 197, 197, 197, 197, 197, 197, 197, 197, 197, 197, 197, 197, 197, 197, 197, 197, 197, 197, 197, 197, 197, 197, 197, 197, 197, 199]
[21537, 22220, 22172, 22476, 21360, 21449, 22392, 21859, 31521, 24969, 24193, 24390, 25403, 24516, 23928, 23264, 24416, 29180, 26844, 25789, 27616, 27319, 30822, 22201, 18114, 17957, 18597, 19484, 18910, 19623, 18932, 23765]


In [8]:
dx_val = X_validation.to_dask_array(lengths=True)
dy_val = y_validation.to_dask_array(lengths=True)

qids_validation = qids_validation.rechunk([tuple(chunks_q)])
dx_val = dx_val.rechunk([tuple(chunks_data), 136])
dy_val = dy_val.rechunk([tuple(chunks_data)])

In [9]:
from lightgbm import DaskLGBMRanker

model = DaskLGBMRanker(
    client=client,
    objective="lambdarank",
    metric="ndcg",
)

model.fit(
    X=dx,
    y=dy,
    group=qids_train,
    eval_set=[(dx_val, dy_val)],
    eval_group=[qids_validation],
    eval_at=5,
    verbose=10,
)

c:\users\silviu\appdata\local\programs\python\python36\lib\site-packages\lightgbm\dask.py:525: UserWarning: Parameter n_jobs will be ignored.
  _log_warning(f"Parameter {param_alias} will be ignored.")


Finding random open ports for workers


LightGBMError: Machine list file doesn't contain the local machine